# Binary Prediction of Poisonous Mushrooms - Modeling

[Competition Link](https://www.kaggle.com/competitions/playground-series-s4e8/data)

Goal of the competition is to predict if a mushroom is poisonous or not based on various mushroom parameters.

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 29/08/2024   | Martin | Created   | Notebook created. Feature engineering and XGBoost | 

# Content

* [Feature Engineering](#feature-engineering)
* [Baseline - XGBoost](#baseline---xgboost)

# Feature Engineering

In [1]:
import os
os.chdir("/tmp/poison_mushrooms")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

## General cleaning

In [3]:
# Remove columns with too many Null
df = df.drop(['id', 'stem-root', 'veil-type', 'veil-color', 'spore-print-color'], axis=1)
df_test = df_test.drop(['stem-root', 'veil-type', 'veil-color', 'spore-print-color'], axis=1)

* https://towardsdatascience.com/deep-embeddings-for-categorical-variables-cat2vec-b05c8ab63ac0
* https://contrib.scikit-learn.org/category_encoders/catboost.html
* https://xgboost.readthedocs.io/en/stable/get_started.html

# Baseline XGBoost

In [20]:
import xgboost as xgb
from xgboost import XGBClassifier

In [5]:
df.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-surface             object
stem-color               object
has-ring                 object
ring-type                object
habitat                  object
season                   object
dtype: object

In [9]:
# Split variables
y = df['class']
X = df.drop('class', axis=1)

mapper = {
  'e': 0,
  'p': 1
}
y = [mapper[i] for i in y]


In [13]:
# Setting categorical variables
for t, col in zip(X.dtypes, X.columns):
  if t == 'object':
    X[col] = X[col].astype("category")

In [17]:
# Define XGBoost model
clf = XGBClassifier(
  tree_method='hist',
  enable_categorical=True,
  device='cuda'
)
clf.fit(X, y)
clf.save_model("models/baseline_xgb.json")

In [25]:
# Predictions
ids = df_test['id']
df_test = df_test.drop('id', axis=1)

# Setting columns
for t, col in zip(df_test.dtypes, df_test.columns):
  if t == 'object':
    df_test[col] = df_test[col].astype("category")

preds = clf.predict(df_test, device='cuda')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:52:47] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [32]:
# Creating output
reverse_mapper = {v: k for k, v in mapper.items()}
result = [reverse_mapper[i] for i in preds]

final = pd.DataFrame({
  'id': ids,
  'class': result
})

final.to_csv('results/baseline_xgb.csv', index=False)

Score on Kaggle: 0.17899